In [1]:
from scipy import sparse
import pandas as pd
import pickle
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
import eli5
import numpy as np
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score,TimeSeriesSplit
import warnings
from IPython.display import display_html
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score,TimeSeriesSplit


In [2]:
train_session = sparse.load_npz('train_session.npz')
test_session = sparse.load_npz('test_session.npz')

In [6]:
train_feats = train_session.tocsr()[:,:-1]
train_feats_coomatrix = sparse.coo_matrix(train_session)
y_train = train_session.tocsr()[:,-1].toarray()

time_split = TimeSeriesSplit(n_splits=10)

In [7]:
xgb_train = train_feats[:int(train_session.shape[0]*0.8),:]
xgb_train_y = y_train[:int(y_train.shape[0]*0.8)]

xgb_test = train_feats[int(train_session.shape[0]*0.8):,:]
xgb_test_y = y_train[int(y_train.shape[0]*0.8):]

eval_set = [(xgb_train,xgb_train_y),(xgb_test, xgb_test_y)]

In [ ]:
# first let's see how much iteration(# of trees) we need for model
eval_metric = ['auc', 'error']
model = XGBClassifier(slient=False,
                     scale_pos_weight=1,
                     learning_rate=0.01,
                     colsample_bytree=0.4,
                     subsample=0.8,
                     objective='binary:logistic',
                     n_estimators=1000,
                     reg_alpha=0.3,
                     max_depth=4,
                     gamma=10)


model.fit(new_train_addYMD, y_train, eval_metric=eval_metric, eval_set=eval_set, verbose=True)

plt.plot(model.evals_result_['validation_0']['auc'],label='train')
plt.plot(model.evals_result_['validation_1']['auc'],label='test')
plt.legend(loc='upper left')
plt.show()

In [ ]:
model = XGBClassifier(slient=False,
                     scale_pos_weight=1,
                     learning_rate=0.01,
                     colsample_bytree=0.4,
                     subsample=0.8,
                     objective='binary:logistic',
                     n_estimators=600,
                     reg_alpha=0.3,
                     gamma=10)

params = {'max_depth': [3,5,7,9,11]}
xgb = GridSearchCV(model, params, scoring='roc_auc', verbose=1, n_jobs=5)
xgb.fit(train_feats_coomatrix,y_train)

C:\Users\900309\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.


In [13]:
xgb.score

<bound method BaseSearchCV.score of GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.4, gamma=10,
       learning_rate=0.01, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=600, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0.3, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, slient=False, subsample=0.8, verbosity=1),
       fit_params=None, iid='warn', n_jobs=5,
       param_grid={'max_depth': [3, 5, 7, 9, 11]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring='roc_auc', verbose=1)>

#### fine tune other components

In [ ]:
params2 = {}